In [1]:
import scipy.optimize
import pandas as pd
import numpy as np
from datetime import datetime as dt

In [2]:
transaction_df = pd.read_csv("transaction.csv")
transaction_df.rename( columns={'Unnamed: 0':'index'}, inplace=True )
transaction_df['due_date']= pd.to_datetime(transaction_df['due_date'])
transaction_df['hire_date']= pd.to_datetime(transaction_df['hire_date'])
transaction_df['return_date']= pd.to_datetime(transaction_df['return_date'])

In [3]:
def process_date(sample):
    hire_date_length = sample.return_date - sample.hire_date
    due_date_length = sample.due_date - sample.hire_date
    ratio = hire_date_length/due_date_length
    # year = int(date_obj[:4])
    # month = int(date_obj[4:6])
    # day = int(date_obj[6:8])
    # sample['year'] = year
    # sample['month'] = month
    sample['test'] = str(hire_date_length).split()[1]
    sample['hire_date_length'] = hire_date_length
    sample['due_date_length'] = due_date_length
    sample['ratio_hire_date'] = round(ratio,3)
    if (ratio > 2).bool():
        sample['point'] = 4.50
    elif (ratio > 1).bool():
        ratio_overdue = abs(1 - ratio)
        sample['point'] = 3 + (7 - (7 * ratio_overdue))
    else:
        sample['test'] = abs((1-ratio)/(ratio-1))
        sample['point'] = 3 + (7*ratio)
    sample['point'] = round(sample['point'],2)
    return sample

In [4]:
transaction_df = transaction_df.groupby('index').apply(process_date).reset_index(drop=True)

In [5]:
transaction_df.head()

,index,hire_date,user_id,book_id,due_date,hire_type,id_y,price,return_date,status,barcode,title_id,test,hire_date_length,due_date_length,ratio_hire_date,point
0,0,2017-02-15,37,53683-70,2017-07-15,2,1,59792.0,2018-03-19 11:25:23,1,53683-70,23,397,397 days 11:25:23,150 days,2.650,4.50
1,1,2017-02-15,39,53398-70,2017-07-15,2,2,100000.0,2017-07-17 10:08:27,1,53398-70,97,152,152 days 10:08:27,150 days,1.016,9.89
2,2,2017-06-06,111,53683-60,2017-06-13,1,3,14948.0,2017-06-12 16:01:18,1,53683-60,23,1,6 days 16:01:18,7 days,0.953,9.67
3,3,2017-07-01,64,58584-80,2017-08-31,2,4,73124.0,2018-03-19 11:17:35,1,58584-80,88,261,261 days 11:17:35,61 days,4.286,4.50
4,4,2017-06-19,25,53664-90,2017-08-31,2,5,64337.0,2017-09-22 14:23:02,1,53664-90,14,95,95 days 14:23:02,73 days,1.310,7.83


In [6]:
transaction_df.columns

Index(['index', 'hire_date', 'user_id', 'book_id', 'due_date', 'hire_type',
       'id_y', 'price', 'return_date', 'status', 'barcode', 'title_id', 'test',
       'hire_date_length', 'due_date_length', 'ratio_hire_date', 'point'],
      dtype='object')

Các điều kiện đánh giá điểm của người dùng dành cho cuốn sách đó:
Ratio = hire_date_length/due_date_length
* Về việc người dùng thuê sách quá hạn:
     - Not finished with the book or books -> 34
* Về việc người dùng thuê sách trả chưa tới hạn trả:
    - Ratio = 0.8 ->   1 | 5*tỷ lệ
    - Ratio = 0.5 -> 0.8 | 


In [7]:
for i in range(0, len(transaction_df)):
    if 1 < transaction_df.iloc[i]['ratio_hire_date'] < 2:
        print(transaction_df.iloc[i]['index'],
              '\t', transaction_df.iloc[i]['book_id'],
              '\t', transaction_df.iloc[i]['hire_date_length'],
              '\t', transaction_df.iloc[i]['hire_date'],
              '\t', transaction_df.iloc[i]['return_date'],
              '\t', transaction_df.iloc[i]['ratio_hire_date'],
              '\t\t', transaction_df.iloc[i]['point'])

1 	 53398-70 	 152 days 10:08:27 	 2017-02-15 00:00:00 	 2017-07-17 10:08:27 	 1.016 		 9.89
4 	 53664-90 	 95 days 14:23:02 	 2017-06-19 00:00:00 	 2017-09-22 14:23:02 	 1.31 		 7.83
5 	 53388-70 	 73 days 14:50:40 	 2017-07-03 00:00:00 	 2017-09-14 14:50:40 	 1.248 		 8.27
10 	 53388-80 	 170 days 16:31:19 	 2017-09-11 00:00:00 	 2018-02-28 16:31:19 	 1.087 		 9.39
14 	 53620-70 	 170 days 10:17:39 	 2017-09-11 00:00:00 	 2018-02-28 10:17:39 	 1.086 		 9.4
19 	 53400-100 	 7 days 12:59:13 	 2017-09-14 00:00:00 	 2017-09-21 12:59:13 	 1.077 		 9.46
20 	 53398-100 	 7 days 12:59:22 	 2017-09-14 00:00:00 	 2017-09-21 12:59:22 	 1.077 		 9.46
23 	 53388-70 	 7 days 14:54:21 	 2017-09-14 00:00:00 	 2017-09-21 14:54:21 	 1.089 		 9.38
25 	 58585-90 	 166 days 13:09:08 	 2017-09-14 00:00:00 	 2018-02-27 13:09:08 	 1.081 		 9.43
26 	 58578-60 	 166 days 13:08:54 	 2017-09-14 00:00:00 	 2018-02-27 13:08:54 	 1.081 		 9.43
58 	 53388-130 	 9 days 11:23:55 	 2017-09-25 00:00:00 	 2017-10-04 11:

In [8]:
print("Overdue with ratio > 2:\t",len(transaction_df[transaction_df.point == 4.50]))
print("Rating < 4.5: \t",len(transaction_df[transaction_df.point < 4.50]))
print("Rating < 3.1: \t",len(transaction_df[transaction_df.point == 3.00]))

Overdue with ratio > 2:	 26
Rating < 4.5: 	 64
Rating < 3.1: 	 0


In [9]:
user_rating = pd.DataFrame()
user_rating['user_id'] = transaction_df.user_id
user_rating['title_id'] = transaction_df.title_id
user_rating['point'] = transaction_df.point

In [10]:
user_rating.head()

,user_id,title_id,point
0,37,23,4.50
1,39,97,9.89
2,111,23,9.67
3,64,88,4.50
4,25,14,7.83


In [11]:
num_user=user_rating.user_id.unique().shape[0]
num_book=user_rating.title_id.unique().shape[0]
print("Number of user: ",num_user)
print("Number of book: ",num_book)
print("Number of record: ",user_rating.shape[0])

Number of user:  108
Number of book:  65
Number of record:  173


In [12]:
def data_rating(sample):
    sample['count'] = str(hire_date_length).split()[1]
    sample['hire_date_length'] = hire_date_length
    sample['due_date_length'] = due_date_length
    sample['ratio_hire_date'] = round(ratio,3)
    if (ratio > 2).bool():
        sample['point'] = 4.50
    elif (ratio > 1).bool():
        ratio_overdue = abs(1 - ratio)
        sample['point'] = 3 + (7 - (7 * ratio_overdue))
    else:
        sample['test'] = abs((1-ratio)/(ratio-1))
        sample['point'] = 3 + (7*ratio)
    sample['point'] = round(sample['point'],2)
    return sample

In [13]:
user_rating_df = user_rating.groupby(['user_id','title_id']).mean().reset_index()

In [14]:
user_rating_df

,user_id,title_id,point
0,8,20,9.430000
1,8,119,9.430000
2,8,140,4.325000
3,9,113,9.080000
4,10,14,9.960000
5,11,164,3.040000
6,13,143,9.520000
7,14,54,4.500000
8,15,66,4.500000
9,16,88,7.863333


In [15]:
def normalize_ratings(my_y, my_r):
    # The mean is only counting movies that were rated
    y_mean = np.sum(my_y, axis=1) / np.sum(my_r, axis=1)
    y_mean = y_mean.reshape((y_mean.shape[0], 1))
    return my_y - y_mean, y_mean

In [16]:
def flatten_params(my_x, my_theta):
    return np.concatenate((my_x.flatten(), my_theta.flatten()))

In [17]:
def reshape_params(flattened_x_and_theta, my_nm, my_nu, my_nf):
    assert flattened_x_and_theta.shape[0] == int(my_nm * my_nf + my_nu * my_nf)
    re_x = flattened_x_and_theta[:int(my_nm * my_nf)].reshape((my_nm, my_nf))
    re_theta = flattened_x_and_theta[int(my_nm * my_nf):].reshape((my_nu, my_nf))
    return re_x, re_theta

In [18]:
def cofi_cost_func(my_params, my_y, my_r, my_nu, my_nm, my_nf, my_lambda=0.):
    my_x, my_theta = reshape_params(my_params, my_nm, my_nu, my_nf)
    term1 = my_x.dot(my_theta.T)
    term1 = np.multiply(term1, my_r)
    cost = 0.5 * np.sum(np.square(term1 - my_y))
    # for regularization
    cost += (my_lambda / 2.) * np.sum(np.square(my_theta))
    cost += (my_lambda / 2.) * np.sum(np.square(my_x))
    return cost

In [19]:
def cofi_grad(my_params, myY, myR, my_nu, my_nm, my_nf, my_lambda=0.):
    my_x, my_theta = reshape_params(my_params, my_nm, my_nu, my_nf)
    term1 = my_x.dot(my_theta.T)
    term1 = np.multiply(term1, myR)
    term1 -= myY
    x_grad = term1.dot(my_theta)
    Thetagrad = term1.T.dot(my_x)
    # Adding Regularization
    x_grad += my_lambda * my_x
    Thetagrad += my_lambda * my_theta
    return flatten_params(x_grad, Thetagrad)

In [27]:
num_user=user_rating_df.user_id.max() #mynu
num_book=user_rating_df.title_id.max()#mynm
print("Number of user: ",num_user)
print("Number of book: ",num_book)
print("Number of record: ",user_rating_df.shape[0])

Number of user:  125
Number of book:  164
Number of record:  145


In [25]:
num_user=user_rating_df.user_id.max #mynu
num_book=user_rating_df.title_id.max #mynm

In [21]:
# Y = np.zeros((mynm, mynu))
# for row in df.itertuples():
#     Y[row[2] - 1, row[4] - 1] = row[3]

matrix_user_book_rating = np.zeros((num_book,num_user))
for row in user_rating.itertuples():
    matrix_user_book_rating[row[2]-1, row[1]-1] = row[3]

In [22]:
R = np.zeros((num_book, num_user))
for i in range(matrix_user_book_rating.shape[0]):
    for j in range(matrix_user_book_rating.shape[1]):
        if matrix_user_book_rating[i][j] != 0:
            R[i][j] = 1
R

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [23]:
Ynorm, Ymean = normalize_ratings(matrix_user_book_rating, R)
X = np.random.rand(num_book, num_user)
Theta = np.random.rand(num_book, num_user)
myflat = flatten_params(X, Theta)
print("myflat:", myflat)
mylambda = 12.2
mynf = 10

myflat: [0.92145685 0.01228048 0.9283781  ... 0.66450751 0.21068896 0.56221498]


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


In [24]:
# result = scipy.optimize.fmin_cg(cofi_cost_func, x0=myflat, fprime=cofi_grad,
    #                                 args=(Y, R, mynu, mynm, mynf, mylambda),
    #                                 maxiter=40, disp=True, full_output=True)
    
result = scipy.optimize.fmin_cg(cofi_cost_func, x0=myflat, fprime=cofi_grad,
                                    args=(matrix_user_book_rating, R, num_user, num_book, mynf, mylambda),
                                    maxiter=40, disp=True, full_output=True)

AssertionError: 